In [23]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from decimal import Decimal

In [73]:
#Define graphing constants
page = (19,10)
single_title = 'Light Intensity of Single Slit Diffraction Pattern'
double_title = 'Light Intensity of Double Slit Interference Pattern'
wavelength = 650 * 10**(-9)     #650nm (red) light
length_s = 0.783                #distance from the sensor aperture to the laser aperture, single slit
length_d = 1.049                #distance from the sensor aperture to the laser aperture, double slit

In [98]:
#Calculate accurate floating point uncertainties using the Decimal module

#Uncertainty percentages recorded from 
#https://www.pasco.com/file_downloads/Downloads_Manuals/PASPORT-High-Sensitivity-Light-Sensor-Manual-PS-2176.pdf
a_slow = 0.0001

''' Finds the first significant digit in Decimal object n.
'''
def firstdigit(n):
    abs_n = abs(n)
    place = 0
    if (abs_n >= Decimal('1.0')):
        while (abs_n >= Decimal('10.0')):
            abs_n = Decimal.shift(abs_n, -1)
            place -= 1
    else:
        while (abs_n < Decimal('1.0')):
            abs_n = Decimal.shift(abs_n, 1)
            place += 1
    return round(n, place)

''' Finds the last significant digit in Decimal object n.
'''
def lastdigit(n):
    place = 0
    while (n % Decimal('1.0') == Decimal('0.0')):
        n = Decimal.shift(n, -1)
    while (n % Decimal('1.0') != Decimal('0.0')):
        n = Decimal.shift(n, 1)
        place -= 1
    return place

''' Calculates the maximum uncertainty by taking the larger between the error of
    accuracy and the error of precision.
    Error of accuracy is rounded to one significant digit.
'''
def finderror(x, a):
    dec_x = Decimal(str(np.abs(x)))
    dec_a = Decimal(str(a))
    err_a = firstdigit(dec_x * dec_a)
    err_p = Decimal('1.0') * Decimal(str(10.0**(lastdigit(dec_x))))
    return (float)(max(err_a, err_p))

In [135]:
''' Saves a graph to directory/filename.png with standard options.
'''
def makegraph(x, y, title, filename, color='red', linewidth=0.4, figsize=page, xlabel='Sensor Position (m)',
              ylabel='Light Intensity (V)', directory='graphs/', show_legend=False, label=None,
              plotsecond=False, secondset=(None, None), secondlabel=None):
    plt.figure(figsize=figsize)
    plt.plot(x, y, color=color, linewidth=linewidth, label=label)
    if plotsecond: plt.plot(secondset[0], secondset[1], color='m', linewidth=linewidth, label=secondlabel)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    if show_legend: plt.legend()
    #plt.show()
    plt.savefig(directory+filename+'.png', bbox_inches='tight')
    plt.clf()

''' Model function for single slit intensity pattern
'''
def intensity_s(x, I, length, a, offset):
    phi = (np.pi * a)/wavelength * ((x - offset)/np.sqrt((x - offset)**2 + length**2))
    return I * (np.sin(phi)/phi)**2

offset = 0.076
length = 1

''' Model function for double slit diffraction pattern
'''
def intensity_d(x, I, a, d):
    phi_s = (np.pi * a)/wavelength * ((x - offset)/np.sqrt((x - offset)**2 + length**2))
    phi_d = (np.pi * d)/wavelength * ((x - offset)/np.sqrt((x - offset)**2 + length**2))
    return I * (np.cos(phi_d))**2 * (np.sin(phi_s)/phi_s)**2

In [136]:
#Load the collected data
single, double, u_s, u_d = [], [], [], []

single.append(np.loadtxt('data/single16a6s10x.txt', skiprows=2))
single.append(np.loadtxt('data/single16a4s1x.txt', skiprows=2))
single.append(np.loadtxt('data/single16a4s100x.txt', skiprows=2))
single.append(np.loadtxt('data/single4a4s100x.txt', skiprows=2))
single.append(np.loadtxt('data/single8a4s100x.txt', skiprows=2))

double.append(np.loadtxt('data/double4a25d4s1x.txt', skiprows=2))
double.append(np.loadtxt('data/double4a25d4s100x.txt', skiprows=2))
double.append(np.loadtxt('data/double4a25d4s100x(2).txt', skiprows=2))
double.append(np.loadtxt('data/double4a50d4s100x.txt', skiprows=2))
double.append(np.loadtxt('data/double8a25d4s100x.txt', skiprows=2))

for i in range(5): 
    u_s.append([finderror(single[i][j,1], a_slow) for j in range(len(single[i]))])
    u_d.append([finderror(double[i][j,1], a_slow) for j in range(len(double[i]))])
    #for j in range(len(double[i])): double[i][j,0] = np.abs(double[i][j,0])
    
#for i in range(5): u_s.append([600 for j in range(len(single[i]))])
#for i in range(5): u_d.append([60 for j in range(len(double[i]))])

single, double, u_s, u_d = np.array(single), np.array(double), np.array(u_s), np.array(u_d)

In [137]:
#Graph the single slit diffraction patterns
makegraph(single[0][:,0], single[0][:,1], single_title+', a=0.16mm', 'single16a6s10x')
makegraph(single[1][:,0], single[1][:,1], single_title+', a=0.16mm', 'single16a4s1x')
makegraph(single[2][:,0], single[2][:,1], single_title+', a=0.16mm', 'single16a4s100x')
makegraph(single[3][:,0], single[3][:,1], single_title+', a=0.4mm', 'single4a4s100x')
makegraph(single[4][:,0], single[4][:,1], single_title+', a=0.8mm', 'single8a4s100x')

#Graph the double slit interference patterns
makegraph(double[0][:,0], double[0][:,1], double_title+', a=0.04mm, d=0.25mm', 'double4a25d4s1x')
makegraph(double[1][:,0], double[1][:,1], double_title+', a=0.04mm, d=0.25mm', 'double4a25d4s100x')
makegraph(double[2][:,0], double[2][:,1], double_title+', a=0.04mm, d=0.25mm', 'double4a25d4s100x(2)')
makegraph(double[3][:,0], double[3][:,1], double_title+', a=0.04mm, d=0.50mm', 'double4a50d4s100x')
makegraph(double[4][:,0], double[4][:,1], double_title+', a=0.08mm, d=0.25mm', 'double8a25d4s100x')

In [138]:
#Setting up constants for curve fit analysis
popt_s, pcov_s = [0] * 5, [0] * 5
popt_d, pcov_d = [0] * 5, [0] * 5

for i in range(5): 
    popt_s[i], pcov_s[i] = curve_fit(intensity_s, single[i][:,0], single[i][:,1], sigma=u_s[i])
popt_s, pcov_s = np.array(popt_s), np.array(pcov_s)

for i in range(5):
    popt_d[i], pcov_d[i] = curve_fit(intensity_d, double[i][:,0], double[i][:,1], sigma=u_d[i])
popt_d, pcov_d = np.array(popt_d), np.array(pcov_d)

print(popt_d)

space = np.linspace(-100.13, 100, 500)
k=0
makegraph(single[k][:,0], single[k][:,1], single_title+', a=0.16mm', 'single16a6s10xfit', plotsecond=True, 
          linewidth=1, secondset=(space, intensity_s(space, popt_s[k,0], popt_s[k,1], popt_s[k,2], popt_s[k,3])), 
          show_legend=True, label='data', secondlabel='best fit')

[[-2.07798524e+07  3.13939620e+01 -3.50696307e+01]
 [ 4.43568793e+06  4.96772160e+01  3.56696762e+01]
 [ 3.06173261e+06  2.45089469e+01 -1.35485612e+01]
 [ 1.35160023e+07  6.98556742e+01  1.00787358e+01]
 [ 1.70791185e+07  5.36171141e+01  1.86650121e+01]]
